# Score Input Images
We use a Caffe CNN model to score HT positive probability for a set of images, outputting a CSV of image SHA1 and positive probability score pairs.



In [ ]:
__depends__ = [
    'eval_image_list.txt',  # List of all evaluation images
    'compute_many_descriptors.json',  #'compute_many_descriptors.json',
    'labels.txt',  # semantic labels for output probability vectors. "positive" must be an option.
]
__dest__ = [
    'image_path_to_sha1.csv',  # output from compute_many_descriptors
    'image_ht_scores.csv',  # Output scores file
]

In [ ]:
import csv
import json

from smqtk.algorithms.classifier.index_label import IndexLabelClassifier
from smqtk.representation.data_element.file_element import DataFileElement
from smqtk.representation.descriptor_index import get_descriptor_index_impls
from smqtk.utils.plugin import from_plugin_config

In [ ]:
IMAGE_LIST_FILE = __depends__[0]
CMD_CONFIG = __depends__[1]
LABELS_FILE = __depends__[2]

IMAGE_PATH_TO_SHA = __dest__[0]
IMAGE_HT_SCORES_CSV = __dest__[1]

In [ ]:
# Compute probability layer results for listed images
!compute_many_descriptors -v -b 1000 --check-image -c $CMD_CONFIG -f $IMAGE_LIST_FILE -p  $IMAGE_PATH_TO_SHA

In [ ]:
# Load configured descriptor index and use with IndexLabelClassifier
cmd_config = json.load(open(CMD_CONFIG))
d_index = from_plugin_config(cmd_config['descriptor_index'], get_descriptor_index_impls())

classifier = IndexLabelClassifier(LABELS_FILE)
descr2cls = classifier.classify_async(d_index, ri=1.0)

In [ ]:
with open(IMAGE_HT_SCORES_CSV, 'w') as f:
    writer = csv.writer(f)
    for cls_elem in descr2cls.itervalues():
        writer.writerow([cls_elem.uuid, cls_elem['positive']])